In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def special_tag(df):
    total = []
    small = []
    numlist = df['number'].tolist()
    taglist = df['specific_tag'].tolist()
    titlelist = df['title'].tolist()
    artistlist = df['artist'].tolist()
    labellist = df['label'].tolist()

    for i in range(len(df)-1):
        if titlelist[i] == titlelist[i+1] and \
           artistlist[i] == artistlist[i+1] and \
          labellist[i] == labellist[i+1]:
            for k in taglist[i]:
                if k in small:
                    continue
                else:
                    small.append(k)
        else:
            for k in taglist[i]:
                if k in small:
                    continue
                else:
                    small.append(k)
            total.append(small)
            small = []

    num1 = numlist[-1]
    #마지막 행 처리
    for j in range(num1):
        for k in taglist[-j-1]:
            if k in small:
                continue
            else:
                small.append(k)

    total.append(small)
    return total

In [ ]:
def remove_duplicate(lst):
    no_duplicate = []
    for i in lst:
        plus = 1
        no_duplicate.append(i)
        while plus < i:
            lst.remove(lst[i+plus-1])
            plus += 1
    return no_duplicate

In [ ]:
def preprocessing(df):
    # 문자열에서 리스트로 변환
    for i in range(len(df)):
        df['specific_tag'][i] = eval(df['specific_tag'][i])

    #number가 1과 1이 아닌 것 구분
    df_1 = df[df['number']==1]
    df_1.drop('tag', axis=1, inplace=True)
    df_2 = df[df['number']!=1]
    df_2.drop('tag', axis=1, inplace=True)

    #number와 title, label로 정렬, 중복 제거된 데이터와 순서가 같아야 함.
    df_2 = df_2.sort_values(by=['number', 'title'])
    unique_pl = df_2[['title','artist', 'label']].value_counts().to_frame(name='number')
    unique_pl = unique_pl.sort_values(by=['number', 'title'])

    #special tag가 합쳐진 리스트 뽑기
    special_taglist = special_tag(df_2)

    #likes 수 중복되지 않게 리스트 만들기
    numlist = df_2['number'].tolist()
    rem_dup_list = remove_duplicate(numlist)
    likelist = df_2['likes'].tolist()
    rem_dup_likelist = []
    plus = 0
    for i in rem_dup_list:
        rem_dup_likelist.append(likelist[plus])
        plus += i

    # unique_pl에 special_tag, likes 붙이기
    unique_pl.reset_index(inplace=True)
    unique_pl['likes'] = rem_dup_likelist
    unique_pl['specific_tag'] = special_taglist
    # 변수 순서 바꿔주기
    unique_pl = unique_pl[['title', 'artist', 'specific_tag', 'label', 'number', 'likes']]

    df_3 = pd.concat([df_1, unique_pl], ignore_index = True)
    return df_3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/bitamin_project/csv/'

In [ ]:
df = pd.read_csv(path+'playlist.csv')
print(len(df))

32423


In [ ]:
df.isnull().sum()

title           0
artist          0
tag             0
specific_tag    0
image           0
file_name       0
label           0
number          0
likes           0
dtype: int64

# title, artist, file_name, image를 추출한 df 생성
중복제거


In [ ]:
tmp = df[['title', 'artist', 'file_name', 'image']].drop_duplicates()
print(len(tmp))
tmp.head()

19081


,title,artist,file_name,image
0,pony,잔나비,pony_잔나비,https://cdnimg.melon.co.kr/cm2/album/images/11...
4,한강에서 (Feat. BIG Naughty),폴킴,한강에서 (Feat. BIG Naughty)_폴킴,https://cdnimg.melon.co.kr/cm2/album/images/11...
11,노래나 부를까,나상현씨밴드,노래나 부를까_나상현씨밴드,https://cdnimg.melon.co.kr/cm2/album/images/11...
13,나의 바람 (Wind And Wish),비투비,나의 바람 (Wind And Wish)_비투비,https://cdnimg.melon.co.kr/cm2/album/images/11...
16,D N D,Apink (에이핑크),D N D_Apink (에이핑크),https://cdnimg.melon.co.kr/cm2/album/images/11...


In [ ]:
df = preprocessing(df.drop(['file_name', 'image'], axis=1)) #number까지 만든 파일 넣기

<ipython-input-20-1fa3dfc5751f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['specific_tag'][i] = eval(df['specific_tag'][i])
<ipython-input-20-1fa3dfc5751f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1.drop('tag', axis=1, inplace=True)
<ipython-input-20-1fa3dfc5751f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.drop('tag', axis=1, inplace=True)


In [ ]:
df.isnull().sum()

title           0
artist          0
specific_tag    0
label           0
number          0
likes           0
dtype: int64

In [ ]:
df = df.sort_values(['title','artist','label'])

copy = df
titlelst = df['title'].tolist()
artistlst = df['artist'].tolist()
labellst = df['label'].tolist()
copy = copy['number'].tolist()
remove_num = []

for i in range(len(df)-1):
    if titlelst[i] == titlelst[i+1] \
    and artistlst[i] == artistlst[i+1] \
    and labellst[i] == labellst[i+1]:
        copy[i] += copy[i+1]
        remove_num.append(i+1)

final = df
final = final.reset_index()
final['number'] = copy
final.drop(remove_num, axis=0, inplace=True)
final.drop(['index'], axis=1, inplace=True)

In [ ]:
final_playlist = pd.merge(final, tmp, on=['title', 'artist'], how='inner')
print(len(final_playlist))
final_playlist.head()

23219


,title,artist,specific_tag,label,number,likes,file_name,image
0,! (느낌표),오늘 (Today),"[미세먼지, 먼지, 저리가, 날려버려, 날씨, 신나는]",foggy,1,907,! (느낌표)_오늘 (Today),https://cdnimg.melon.co.kr/cm/album/images/022...
1,"""Méditation"" from Thaïs",Joshua Bell,"[클래식, 여름, 연주곡, 장마, 휴식]",rainy,1,7,%2Méditation%2 from Thaïs_Joshua Bell,https://cdnimg.melon.co.kr/cm2/album/images/11...
2,#LoveSTAY,Stray Kids (스트레이 키즈),"[노을, 케이팝, 감성, 해질녘, 잔잔한, 휴식, 힐링, 한강, 아이돌, 저녁]",sunrise_sunset,1,6837,#LoveSTAY_Stray Kids (스트레이 키즈),https://cdnimg.melon.co.kr/cm2/album/images/11...
3,#menow,프로미스나인,"[가요, 여름, 여행, 기분전환, 드라이브, 휴가, 시원한, 청량한, 더위, 명곡]",sunny,1,15670,#menow_프로미스나인,https://cdnimg.melon.co.kr/cm2/album/images/11...
4,#봄,꽃가람,"[비오는날, 봄비, 비노래, 봄, 봄노래, 그리움, 비내리는날, 여름비, 설레임]",rainy,1,13,#봄_꽃가람,https://cdnimg.melon.co.kr/cm/album/images/101...


In [ ]:
# 데이터 프레임 저장
final_playlist.to_csv(path+'final_playlist.csv', encoding='utf-8-sig',index=False)